# Database Governance Quickstart

[!["Open In Colab"](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/01_quickstart/02_database_governance.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/01_quickstart/02_database_governance.ipynb)

## 🏢 Business Scenario
Most organizations have critical data locked in traditional **SQL Databases** (ERP, CRM, Legacy apps). To unlock analytics and ML, this data needs to move to a **Modern Data Lakehouse**.

**The Problem**: Database entries are often messy (null emails, invalid IDs, duplicate entries). If you just copy-paste this data into your lake, you're building a "Data Swamp."

**The Solution**: LakeLogic acts as a **Governance Gate**. It extracts the data but automatically redirects messy records to a Quarantine (layer), ensuring only "Gold Standard" data reaches your analytics-ready Silver layer.

## 🎯 What You'll Learn
1. Setting up a "Dirty" source database
2. Defining a **Data Contract** with Quality Rules
3. Using LakeLogic to extract, validate, and protect your Lakehouse
4. Inspecting the Quality split (Good vs. Bad data)

## 🚀 Step 1: Setup Local Database (with Dirty Data)
We'll use **SQLite** to simulate a source database. We'll intentionally insert 1 good record and 2 records that violate our business rules.

In [1]:
import sqlite3
import os

db_file = 'example.db'
conn = sqlite3.connect(db_file)
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS users (id INTEGER, name TEXT, email TEXT)')
c.execute('DELETE FROM users')

# 1. Good Record
c.execute('INSERT INTO users VALUES (1, "Alice", "alice@example.com")')

# 2. Bad Record (Invalid Email format)
c.execute('INSERT INTO users VALUES (2, "Bob", "bob-invalid-email-format")')

# 3. Bad Record (Impossible ID)
c.execute('INSERT INTO users VALUES (-99, "Charlie", "charlie@example.com")')

conn.commit()
conn.close()
print("✅ Local database setup complete with 'Dirty Data' (1 Good, 2 Bad)!")

✅ Local database setup complete with 'Dirty Data' (1 Good, 2 Bad)!


## 📝 Step 2: Use Your Contract
We have a `users_contract.yaml` file that defines our desired schema and **Quality Rules**. LakeLogic will use these rules to filter out the bad records automatically.

In [2]:
with open('users_contract.yaml', 'r') as f:
    print("📄 users_contract.yaml content (Note the 'quality' section!):")
    print("-----------------------------")
    print(f.read())

📄 users_contract.yaml content (Note the 'quality' section!):
-----------------------------
dataset: remote_employees
quality:
  row_rules:
  - name: Valid Email
    sql: email LIKE '%@%'
source:
  type: landing
version: 1.0.0



## ?? Step 3: Run the Governance Pipeline
LakeLogic validates a dataframe, so we?ll extract from SQLite with the standard library and
pass the rows into `DataProcessor.run()`.

We?ll use **Named Attribute access** (`result.good`, `result.bad`) for maximum readability.


In [3]:
from lakelogic import DataProcessor
import sqlite3

# Extract rows from SQLite using the standard library
conn = sqlite3.connect("example.db")
conn.row_factory = sqlite3.Row
rows = conn.execute("SELECT * FROM users").fetchall()
conn.close()
data = [dict(row) for row in rows]

# Initialize processor with our contract path
processor = DataProcessor(contract="users_contract.yaml")

# Run the validation pipeline
result = processor.run(data)

print(f"Run Complete via {processor.engine_name} engine!")
print(f"   Records Validated: {len(result.good)}")
print(f"   Records Quarantined: {len(result.bad)}")


2026-02-21 10:20:41.794 | INFO     | lakelogic.core.processor:run:349 - Run complete. Source: 3, Total (post-transform): 3, Good: 2, Quarantined: 1, Pre-Transform Dropped: 0, Ratio: 33.33%


Run Complete via polars engine!
   Records Validated: 2
   Records Quarantined: 1


## 🎉 Inspect the Results
Let's see the power of LakeLogic: separating the "clean" data from the "corrupt" data.

In [4]:
print("🚨 QUARANTINE (Rejected DB Rows):")
print(result.bad)

print("\n🏆 CLEAN SILVER DATA (Analytics Ready):")
print(result.good)

🚨 QUARANTINE (Rejected DB Rows):
shape: (1, 7)
┌─────┬──────┬─────────────────┬────────────────┬────────────────┬────────────────┬────────────────┐
│ id  ┆ name ┆ email           ┆ _lakelogic_err ┆ _lakelogic_cat ┆ quarantine_sta ┆ quarantine_rep │
│ --- ┆ ---  ┆ ---             ┆ ors            ┆ egories        ┆ te             ┆ rocessed       │
│ i64 ┆ str  ┆ str             ┆ ---            ┆ ---            ┆ ---            ┆ ---            │
│     ┆      ┆                 ┆ list[str]      ┆ list[str]      ┆ str            ┆ bool           │
╞═════╪══════╪═════════════════╪════════════════╪════════════════╪════════════════╪════════════════╡
│ 2   ┆ Bob  ┆ bob-invalid-ema ┆ ["Rule failed: ┆ ["correctness" ┆ active         ┆ false          │
│     ┆      ┆ il-format       ┆ Valid Email    ┆ ]              ┆                ┆                │
│     ┆      ┆                 ┆ (em…           ┆                ┆                ┆                │
└─────┴──────┴─────────────────┴────────────

## 🏁 Summary
Instead of writing complex validation scripts for every table, you defined a **Data Contract**. LakeLogic ensured that only Alice reached the Lakehouse, while Bob and Charlie were safely isolated for review.

This is **Shift-Left Security** for your Data Lakehouse.